----

# **Anomaly Detection & Forecasting**


----

In [ ]:
## To do
# send e-mail if something wrong?

## **1. Imports**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import StandardScaler, normalize
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest
import pickle
from prophet import Prophet

## **2. Load and read file**

In [ ]:
df_inputx = pd.read_csv("input.csv")
df_inputx.head()

In [ ]:
### filter out EXTSPT01
df_input = df_inputx[df_inputx['EXTRACT']=='EXTSPT02']

In [ ]:
#size of df
print("Shape of Dataframe" + str(df_input.shape))

In [ ]:
#check the data type per column
df_input.dtypes

In [ ]:
#review unique values
print(df_input['EXTRACT'].unique())
print(df_input['DATA_PUMP'].unique())
print(df_input['REPLICAT'].unique())

In [ ]:
#review input data
df_input.describe()

In [ ]:
df_inputx = df_input.head(1000)
xpoints = df_inputx['SOURCE_HB_TS']
ypoints = df_inputx['REPLICAT_READ_LAG']

plt.figure().set_figwidth(15)
plt.plot(xpoints, ypoints, linestyle = 'dotted')
plt.xlabel('TIME')
plt.ylabel('REPLICAT_READ_LAG')

#plt.xticks(rotation=45)

# Display the plot
plt.show()

In [ ]:
df_inputx = df_input.head(10000)
xpoints = df_inputx['SOURCE_HB_TS']
ypoints = df_inputx['EXTRACT_LAG']

plt.figure().set_figwidth(15)
plt.plot(xpoints, ypoints, linestyle = 'dotted')
plt.xlabel('TIME')
plt.ylabel('EXTRACT_LAG')

#plt.xticks(rotation=45)

# Display the plot
plt.show()

In [ ]:
# df_inputx = df_input.head(10000)
# xpoints = df_inputx['SOURCE_HB_TS']
# ypoints = df_inputx['DATA_PUMP_READ_LAG']

# plt.figure().set_figwidth(15)
# plt.plot(xpoints, ypoints, linestyle = 'dotted')
# plt.xlabel('TIME')
# plt.ylabel('DATA_PUMP_READ_LAG')

# #plt.xticks(rotation=45)

# # Display the plot
# plt.show()

In [ ]:
# df_inputx = df_input.head(10000)
# xpoints = df_inputx['SOURCE_HB_TS']
# ypoints = df_inputx['REPLICAT_READ_LAG']

# plt.figure().set_figwidth(15)
# plt.plot(xpoints, ypoints, linestyle = 'dotted')
# plt.xlabel('TIME')
# plt.ylabel('REPLICAT_READ_LAG')

# #plt.xticks(rotation=45)

# # Display the plot
# plt.show()

---

# **3. Data Prep**

In [ ]:
#only get 5 individual columns
df_input_kpi = df_input[['SOURCE_HB_TS', 'EXTRACT_LAG', 'DATA_PUMP_READ_LAG', 'REPLICAT_READ_LAG', 'REPLICAT_APPLY_LAG', 'TOTAL_LAG']]

#convert object to datetime
df_input_kpi['SOURCE_HB_TS'] = pd.to_datetime(df_input_kpi['SOURCE_HB_TS'])
df_input_kpi.head()

In [ ]:
#check datatime. First column should be datetime
df_input_kpi.dtypes

# **4. ML models**

In [ ]:
last_minutes_used_in_training = 60  #30 minutes used for training
forecast_in_minutes = 10  #10 minutes will be forecast

In [ ]:
print("Dataframe ends at " + str(df_input_kpi.tail(1).SOURCE_HB_TS))

## Model 1 - FB Prophet

In [ ]:
#filter on columsn and change name
df_input_extract_lag = df_input[['SOURCE_HB_TS', 'TOTAL_LAG']]

df_input_extract_lag.rename(columns={"SOURCE_HB_TS": "ds", "TOTAL_LAG":"y"}, inplace=True)
df_input_extract_lag['ds'] = pd.to_datetime(df_input_extract_lag['ds'])

#filter on X last minutes for forecast
input_prophet_1 = df_input_extract_lag.tail(last_minutes_used_in_training)
input_prophet_1['ds'] = pd.to_datetime(input_prophet_1['ds'])

m_1 = Prophet(changepoint_prior_scale=0.9, daily_seasonality=True)
m_1.fit(input_prophet_1)

In [ ]:
#create a empty dataframe with forecast dates
future_1 = m_1.make_future_dataframe(periods=forecast_in_minutes, freq="min")

#use the model to predict
forecast_1 = m_1.predict(future_1)
forecast_1[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
fig1 = m_1.plot(forecast_1)

### Use the forecast (10 minutes in the future) to see expected growth

In [ ]:
## simple calculation as example

# average actual 30 minutes as input
avg_input = forecast_1['yhat'].head(30).mean()
max_input = forecast_1['yhat'].head(30).max()
min_input = forecast_1['yhat'].head(30).min()
print("Average in input value is " + str(avg_input))
print("Max input value " + str(max_input))
print("Min input value " + str(min_input))
print(" ------------------- ")
#average 10 minutes as forecasted
avg_forecast = forecast_1['yhat'].tail(10).mean()
max_forecast = forecast_1['yhat'].tail(10).max() 
min_forecast = forecast_1['yhat'].tail(10).min() 
print("Average in forecast value is " + str(avg_forecast))
print("Max forecast value " + str(max_forecast))
print("Min forecast value " + str(min_forecast))
print(" ------------------- ")

#increase/decrease based on actual vs forecasted
diff_perc_input_vs_forecast = round(((avg_forecast-avg_input)/avg_input)*100,2)
diff_perc_max = round(((max_forecast-max_input)/max_input)*100,2)
diff_perc_min = round(((min_forecast-min_input)/min_input)*100,2)
print("Expected increase or decreases in the coming 10 minutes is " + str(diff_perc_input_vs_forecast)+str("%"))
print("Expected increase/decrease in max "  + str(diff_perc_max)+str("%"))
print("Expected increase/decrease in min "  + str(diff_perc_min)+str("%"))


---

# **4. Create model artifacts**

In [ ]:
#!odsc conda init -b conda_env -n frzpemb9ufe8 -a resource_principal

In [ ]:
#!odsc conda publish -s tensorflow28_p38_cpu_v1 --force

In [189]:
from ads.model.framework.tensorflow_model import TensorFlowModel
from ads.common.model_metadata import UseCaseType
from ads.common.model_artifact import ModelArtifact
from ads.common.model_export_util import prepare_generic_model
import os

In [190]:
#path to artifacts and conda slug
path_to_artifacts = './model_artifacts_v8'
conda_env = 'oci://conda_env@frzpemb9ufe8/conda_environments/cpu/TensorFlow 2.8 for CPU on Python 3.8/1.0/tensorflow28_p38_cpu_v1'  

#create default artifacts
artifact = prepare_generic_model(
    path_to_artifacts, 
    fn_artifact_files_included=False, 
    force_overwrite=True, 
    inference_conda_env=conda_env)

  artifact = prepare_generic_model(



loop1:   0%|          | 0/4 [00:00<?, ?it/s]

# **5. Full code score.py**

In [191]:
%%writefile ./model_artifacts_v8/score.py

import pandas as pd
import numpy as np
import uuid
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import StandardScaler, normalize
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest
import pickle
import gzip
from prophet import Prophet
import ads
import os
import configparser
import shutil
from zipfile import ZipFile
from tempfile import NamedTemporaryFile
import urllib
import re
import sqlalchemy
from sqlalchemy import create_engine
import cx_Oracle
from ocifs import OCIFileSystem
import cx_Oracle


def load_model():
    class DummyModel:
        def __init__(self):
            pass
    return DummyModel()

#create folder for input files
if not os.path.exists("input_files"):
    os.makedirs("input_files")

############################
############################


list_variables = ['EXTRACT_LAG', 'DATA_PUMP_READ_LAG', 'REPLICAT_READ_LAG', 'REPLICAT_APPLY_LAG', 'TOTAL_LAG']



def predict(data, model=load_model()):
    
    #test print root
    print('Get current working directory : ', os.getcwd())
       
    input_files_location = os.getcwd() + "/input_files/"
    print("Input file full location " + input_files_location)

    #get the bucket name, namespace, and full file name
    file_name = data['file_name']
    bucket_name = data['bucket_name']
    namespace = data['namespace']
    
    #get full location in bucket
    full_location_in_bucket = "oci://" + bucket_name + "@" + namespace + "/LagMetrics/ocid1.serviceconnector.oc1.eu-frankfurt-1.amaaaaaapixtsjiarpunoxhi6tvcw3jdbgwyt6xzv4cl4zryhuigxesyyjmq/"+file_name
    print("full location in bucket " + full_location_in_bucket)
       
    raw_input_from_zip = pd.read_csv(full_location_in_bucket, names=['SOURCE_HB_TS','EXTRACT', 'EXTRACT_LAG','DATA_PUMP','DATA_PUMP_READ_LAG','REPLICAT','REPLICAT_READ_LAG','REPLICAT_APPLY_LAG','TOTAL_LAG'], header=None)
    
    #read the .txt file
    input_all_minutes = raw_input_from_zip
    print("first line of data" + str(input_all_minutes.head(1)))
    
    #get the latest 60 minues only.
    input_60_minutes = input_all_minutes.tail(60)
    print(input_60_minutes.shape)
    
    #add random id for id set
    set_id = "set_id_"+ str(uuid.uuid4())
    
        
    #loop through the 5 variables. Build forecast for each one of them and push to database
    for variable in list_variables:
        
        print("-------------------------------------------------------")
        print("Start variable " + variable)
        print(type(variable))
        
        #create empty list
        list_to_db =  []
    
        #only select one variable to filter on
        df_input_1_variable = input_60_minutes[['SOURCE_HB_TS', variable]]
        
        #change name to what Prophet expects       
        df_input_1_variable.rename(columns={"SOURCE_HB_TS": "ds", variable:"y"}, inplace=True)
        
        #convert ds to timeframe
        df_input_1_variable['ds'] = pd.to_datetime(df_input_1_variable['ds'])
        
        #start date
        start_date = df_input_1_variable['ds'].min()
        print("Start time of " + variable + " is " + str(start_date))

        #end date
        end_date = df_input_1_variable['ds'].max()
        print("End time of " + variable + " is " + str(end_date))

        #general settings
        forecast_in_minutes = 10  #10 minutes will be forecast

        m_1 = Prophet(changepoint_prior_scale=0.9)
        m_1.fit(df_input_1_variable)

        #create a empty dataframe with forecast dates
        future_1 = m_1.make_future_dataframe(periods=forecast_in_minutes, freq="min")

        #use the model to predict
        forecast_1 = m_1.predict(future_1)
        forecast_1[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

        # average actual 60 minutes as input
        avg_input = forecast_1['yhat'].head(60).mean()
        max_input = forecast_1['yhat'].head(60).max()
        min_input = forecast_1['yhat'].head(60).min()
        print("Average in input value is " + str(avg_input))
        print("Max input value " + str(max_input))
        print("Min input value " + str(min_input))
        print(" ------------------- ")

        #average 10 minutes as forecasted
        avg_forecast = forecast_1['yhat'].tail(10).mean()
        max_forecast = forecast_1['yhat'].tail(10).max() 
        min_forecast = forecast_1['yhat'].tail(10).min() 
        print("Average in forecast value is " + str(avg_forecast))
        print("Max forecast value " + str(max_forecast))
        print("Min forecast value " + str(min_forecast))
        print(" ------------------- ")

        #increase/decrease based on actual vs forecasted
        diff_perc_avg = round(((avg_forecast-avg_input)/avg_input)*100,2)
        diff_perc_max = round(((max_forecast-max_input)/max_input)*100,2)
        diff_perc_min = round(((min_forecast-min_input)/min_input)*100,2)
        print("Expected increase or decreases in the coming 10 minutes is " + str(diff_perc_avg)+str("%"))
        print("Expected increase/decrease in max "  + str(diff_perc_max)+str("%"))
        print("Expected increase/decrease in min "  + str(diff_perc_min)+str("%"))
        
        
        #add to list
        list_to_db.append([set_id, variable, start_date, end_date, avg_input, max_input, min_input, avg_forecast, max_forecast, min_forecast, diff_perc_avg, diff_perc_max, diff_perc_min])
        df_to_db = pd.DataFrame(list_to_db, columns =['set_id', 'variable', 'start_date', 'end_date', 'avg_input', 'max_input', 'min_input','avg_forecast', 'max_forecast', 'min_forecast', 'diff_perc_avg', 'diff_perc_max', 'diff_perc_min'])

        #################
        ################ push results to adw
       
        connection_parameters = {
        'user_name': 'OMLUSER',
        'password': 'WElcome11POC##',
        'service_name': 'pocdb_high',
        'wallet_location': "/home/datascience/model-server/app/deployed_model/credentials/Wallet_pocdb.zip",
        }

        
        ## push results to database
        df_to_db.ads.to_sql('maersk_logs_v3', connection_parameters=connection_parameters, if_exists="append")
                            
                            
                            #, dtype={
                
#                 'variable': sqlalchemy.types.NVARCHAR(length = 500),
#                 'start_date': sqlalchemy.types.DateTime(),
#                 'end_date': sqlalchemy.types.DateTime(),
#                 'avg_input': sqlalchemy.types.FLOAT(),
#                 'max_input': sqlalchemy.types.FLOAT(),
#                 'min_input': sqlalchemy.types.FLOAT(),
#                 'avg_forecast': sqlalchemy.types.FLOAT(),
#                 'max_forecast': sqlalchemy.types.FLOAT(),            
#                 'min_forecast': sqlalchemy.types.FLOAT(),            
#                 'diff_perc_avg': sqlalchemy.types.FLOAT(),
#                 'diff_perc_max': sqlalchemy.types.FLOAT(),
#                 'diff_perc_min': sqlalchemy.types.FLOAT()})

        #delete list for next loop
        del list_to_db
        
        print()
        print("-----------------------")
        print("Table updated with results for " + variable)
        print("-----------------------")
    
    
    #return {'diff_perc_input_vs_forecast':diff_perc_input_vs_forecast, 'avg_input':avg_input, 'avg_forecast':avg_forecast}

Overwriting ./model_artifacts_v8/score.py


# **6. Test the full code with JSON as input**

In [187]:
# #example input

data = {'file_name':'20231017T103618Z_20231017T104258Z.0.log.gz', 'bucket_name':'LagMetricFiles', 'namespace':'frzpemb9ufe8'}


In [188]:
predict(data)

Get current working directory :  /home/datascience
Input file full location /home/datascience/input_files/
full location in bucket oci://LagMetricFiles@frzpemb9ufe8/LagMetrics/ocid1.serviceconnector.oc1.eu-frankfurt-1.amaaaaaapixtsjiarpunoxhi6tvcw3jdbgwyt6xzv4cl4zryhuigxesyyjmq/20231017T103618Z_20231017T104258Z.0.log.gz
first line of data                SOURCE_HB_TS   EXTRACT  EXTRACT_LAG  \
0  22-SEP-23 15:27:30.013717  EXTSPT01     3.118273   

                      DATA_PUMP  DATA_PUMP_READ_LAG REPLICAT  \
0  scrbgcsdk011660:9021:SPTFC01            0.523514    RSPT1   

   REPLICAT_READ_LAG  REPLICAT_APPLY_LAG  TOTAL_LAG  
0           0.281207            0.146456    4.06945  
(60, 9)
-------------------------------------------------------
Start variable EXTRACT_LAG
<class 'str'>
Start time of EXTRACT_LAG is 2023-09-23 09:12:30.037776
End time of EXTRACT_LAG is 2023-09-23 09:42:30.010488
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override t

07:55:54 - cmdstanpy - INFO - Chain [1] start processing


INFO:cmdstanpy:Chain [1] start processing


07:55:54 - cmdstanpy - INFO - Chain [1] done processing


INFO:cmdstanpy:Chain [1] done processing
Average in input value is 2.9615368188352806
Max input value 3.146609116439366
Min input value 2.6079613863678617
 ------------------- 
Average in forecast value is 2.723408001576514
Max forecast value 2.8388546167851665
Min forecast value 2.6079613863678617
 ------------------- 
Expected increase or decreases in the coming 10 minutes is -8.04%
Expected increase/decrease in max -9.78%
Expected increase/decrease in min 0.0%

-----------------------
Table updated with results for EXTRACT_LAG
-----------------------
-------------------------------------------------------
Start variable DATA_PUMP_READ_LAG
<class 'str'>
Start time of DATA_PUMP_READ_LAG is 2023-09-23 09:12:30.037776
End time of DATA_PUMP_READ_LAG is 2023-09-23 09:42:30.010488
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:pro

...

INFO:cmdstanpy:Chain [1] done processing
Average in input value is 0.5496867328923013
Max input value 0.6719069600978214
Min input value 0.21251572081599998
 ------------------- 
Average in forecast value is 0.6556374746301584
Max forecast value 0.6719069600978214
Min forecast value 0.6393679891624955
 ------------------- 
Expected increase or decreases in the coming 10 minutes is 19.27%
Expected increase/decrease in max 0.0%
Expected increase/decrease in min 200.86%

-----------------------
Table updated with results for REPLICAT_APPLY_LAG
-----------------------
-------------------------------------------------------
Start variable TOTAL_LAG
<class 'str'>
Start time of TOTAL_LAG is 2023-09-23 09:12:30.037776
End time of TOTAL_LAG is 2023-09-23 09:42:30.010488
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling d

07:55:55 - cmdstanpy - INFO - Chain [1] start processing


INFO:cmdstanpy:Chain [1] start processing


07:55:56 - cmdstanpy - INFO - Chain [1] done processing


INFO:cmdstanpy:Chain [1] done processing
Average in input value is 4.54157445725518
Max input value 4.832784914949162
Min input value 3.991049017696123
 ------------------- 
Average in forecast value is 4.171208565264914
Max forecast value 4.351368112833706
Min forecast value 3.991049017696123
 ------------------- 
Expected increase or decreases in the coming 10 minutes is -8.16%
Expected increase/decrease in max -9.96%
Expected increase/decrease in min 0.0%

-----------------------
Table updated with results for TOTAL_LAG
-----------------------


In [ ]:
#data = {'file_name':'20231017T103618Z_20231017T104258Z.0.log.gz', 'bucket_name':'LagMetricFiles', 'namespace':'frzpemb9ufe8'}



In [192]:
#all should be passed
artifact.introspect()

['score.py', 'input_files', '.ipynb_checkpoints', 'test_json_output.json', 'credentials', 'runtime.yaml']


,Test key,Test name,Result,Message
0,runtime_env_path,Check that field MODEL_DEPLOYMENT.INFERENCE_ENV_PATH is set,Passed,
1,runtime_env_python,Check that field MODEL_DEPLOYMENT.INFERENCE_PYTHON_VERSION is set to a value of 3.6 or higher,Passed,
2,runtime_path_exist,Check that the file path in MODEL_DEPLOYMENT.INFERENCE_ENV_PATH is correct.,Passed,
3,runtime_version,Check that field MODEL_ARTIFACT_VERSION is set to 3.0,Passed,
4,runtime_yaml,"Check that the file ""runtime.yaml"" exists and is in the top level directory of the artifact directory",Passed,
5,score_load_model,Check that load_model() is defined,Passed,
6,score_predict,Check that predict() is defined,Passed,
7,score_predict_arg,Check that all other arguments in predict() are optional and have default values,Passed,
8,score_predict_data,"Check that the only required argument for predict() is named ""data""",Passed,
9,score_py,"Check that the file ""score.py"" exists and is in the top level directory of the artifact directory",Passed,


In [193]:
# Saving the model artifact to the model catalog. 
import ads
ads.set_auth(auth='resource_principal')

catalog_entry = artifact.save(display_name='maersk_logs_v8', description='maersk_logs_v8', timeout=600)
catalog_entry.id

loop1:   0%|          | 0/5 [00:00<?, ?it/s]

'ocid1.datasciencemodel.oc1.eu-frankfurt-1.amaaaaaapixtsjia3z7z2spuadqer3dxtxjvagnd6rbuyn2qfwvpg5xu6hhq'

# **7. Deploy ML Model**

# **8. Test Deployed ML Model - Version 1**

In [145]:
import requests
import oci
from oci.signer import Signer
import json

In [146]:
%%time
url = "https://modeldeployment.eu-frankfurt-1.oci.customer-oci.com/ocid1.datasciencemodeldeployment.oc1.eu-frankfurt-1.amaaaaaapixtsjiasf2ieflxolc66jkhugoklfsn5ldkvlizigc4qt65arla/predict"

#data = {'file_name':'logs.csv', 'bucket_name':'LagMetricFiles', 'namespace':'frzpemb9ufe8'}
data = {'file_name':'20231018T004336Z_20231018T005033Z.0.log.gz', 'bucket_name':'LagMetricFiles', 'namespace':'frzpemb9ufe8'}

auth = oci.auth.signers.get_resource_principals_signer()

#POST request to the model
response = requests.post(url, json=data, auth=auth)
print(response)
print(json.loads(response.content))

full_response = json.loads(response.content)

<Response [200]>
None
CPU times: user 56.7 ms, sys: 1.41 ms, total: 58.1 ms
Wall time: 5.63 s


## **Multiple requests - Version 1**

In [ ]:
#example input
import time
df_input_extract_lag = df_input[['SOURCE_HB_TS', 'REPLICAT_APPLY_LAG']]
df_input_extract_lag.rename(columns={"SOURCE_HB_TS": "ds", "REPLICAT_APPLY_LAG":"y"}, inplace=True)

rotations = [40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50]

for setx in rotations:
    #set of 30 minutes
    input_prophet_1 = df_input_extract_lag[setx:setx+30]  #last 30 minutes, like: 40 - 70, 41 - 71, etc
    
    #convert to json
    data = input_prophet_1.to_json()
    
    url = "https://modeldeployment.eu-frankfurt-1.oci.customer-oci.com/ocid1.datasciencemodeldeployment.oc1.eu-frankfurt-1.amaaaaaapixtsjiaevm3vjjcyfxf2xr2k5ecrfobeo5eog5v2exoli7r4o3q/predict"

    auth = oci.auth.signers.get_resource_principals_signer()

    #POST request to the model
    response = requests.post(url, json=data, auth=auth)
    full_response = json.loads(response.content)
    print()
    print("XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX")
    print("Set number is " + str(setx))
    print("**Predicted increase or decrease in the coming 10 minutes = **" +str(full_response['diff_perc_input_vs_forecast']))
    print("Average number last 30 minutes = " + str(full_response['avg_input']))
    print("Average forecasted number future 10 minutes = " + str(full_response['avg_forecast']))
    print("XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX")
    print()
    time.sleep(2)



## **Multiple requests - Version 2**

In [ ]:
#example input
import time
df_input_extract_lag = df_input[['SOURCE_HB_TS', 'REPLICAT_APPLY_LAG']]
df_input_extract_lag.rename(columns={"SOURCE_HB_TS": "ds", "REPLICAT_APPLY_LAG":"y"}, inplace=True)

rotations = [40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50]

for setx in rotations:
    #set of 30 minutes
    input_prophet_1 = df_input_extract_lag[setx:setx+30]  #last 30 minutes, like: 40 - 70, 41 - 71, etc
    
    #convert to json
    data = input_prophet_1.to_json()
    
    url = "https://modeldeployment.eu-frankfurt-1.oci.customer-oci.com/ocid1.datasciencemodeldeployment.oc1.eu-frankfurt-1.amaaaaaapixtsjian3qrkhcw5wazrbquy7nbovnqnrzi2cpr45ikbseywr6a/predict"

    auth = oci.auth.signers.get_resource_principals_signer()

    #POST request to the model
    response = requests.post(url, json=data, auth=auth)
    full_response = json.loads(response.content)
    print()
    print("XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX")
    print("Set number is " + str(setx))
    print("**Predicted increase or decrease in the coming 10 minutes = **" +str(full_response['diff_perc_input_vs_forecast']))
    print("Average number last 30 minutes = " + str(full_response['avg_input']))
    print("Average forecasted number future 10 minutes = " + str(full_response['avg_forecast']))
    print("XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX")
    print()
    time.sleep(3)

# **Python script in Function**

In [6]:
# def get_object(bucketName, objectName):
#     signer = oci.auth.signers.get_resource_principals_signer()
#     client = oci.object_storage.ObjectStorageClient(config={}, signer=signer)
#     namespace = client.get_namespace().data
#     try:
#         print("Searching for bucket and object", flush=True)
#         object = client.get_object(namespace, bucketName, objectName)
#         print("found object", flush=True)
#         if object.status == 200:
#             print("Success: The object " + objectName + " was retrieved with the content: " + object.data.text, flush=True)
#             message = object.data.text
#         else:
#             message = "Failed: The object " + objectName + " could not be retrieved."
#     except Exception as e:
#         message = "Failed: " + str(e.message)
#     return { "content": message }

In [7]:
## example data from Events to Functions

data = {
  "cloudEventsVersion": "0.1",
  "eventID": "unique_ID",
  "eventType": "com.oraclecloud.objectstorage.createobject",
  "source": "objectstorage",
  "eventTypeVersion": "2.0",
  "eventTime": "2019-01-10T21:19:24.000Z",
  "contentType": "application/json",
  "extensions": {
    "compartmentId": "ocid1.compartment.oc1..unique_ID"
  },
  "data": {
    "compartmentId": "ocid1.compartment.oc1..unique_ID",
    "compartmentName": "example_name",
    "resourceName": "my_object",
    "resourceId": "/n/example_namespace/b/my_bucket/o/my_object",
    "availabilityDomain": "all",
    "additionalDetails": {
      "eTag": "f8ffb6e9-f602-460f-a6c0-00b5abfa24c7",
      "namespace": "example_namespace",
      "bucketName": "my_bucket",
      "bucketId": "ocid1.bucket.oc1.phx.unique_id",
      "archivalState": "Available"
    }
  }
}


In [ ]:
data['data']['resourceName']

In [ ]:
import oci
from datetime import datetime, timezone
import requests
from oci.signer import Signer
import json

auth = oci.auth.signers.get_resource_principals_signer()
object_storage_client = oci.object_storage.ObjectStorageClient(config={}, signer=signer)

def handler(ctx, data: io.BytesIO = None):
    
    bucket_name = "LagMetricFiles"
    namespace = "frzpemb9ufe8"
    #sub_bucket = "LagMetrics/ocid1.serviceconnector.oc1.eu-frankfurt-1.amaaaaaapixtsjiarpunoxhi6tvcw3jdbgwyt6xzv4cl4zryhuigxesyyjmq"
    
    latest_file_name = data['data']['resourceName']

    #cal the HTTP endpoint
    data = {'file_name':latest_file_name, 'bucket_name':bucket_name, 'namespace':namespace}

    url = "https://modeldeployment.eu-frankfurt-1.oci.customer-oci.com/ocid1.datasciencemodeldeployment.oc1.eu-frankfurt-1.amaaaaaapixtsjia7wjigtlgiqvy652ymano5nrfcpsf7piptjx77s2xylia/predict"

    response = requests.post(url, json=data, auth=auth)
    full_response = json.loads(response.content)
    print(full_response) #for logs


In [ ]:
data = {'file_name':'20231017T103618Z_20231017T104258Z.0.log.gz', 'bucket_name':'LagMetricFiles', 'namespace':'frzpemb9ufe8'}